In [1]:
import os
os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"  
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"   
import time
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

In [2]:
print("Built with CUDA:", tf.test.is_built_with_cuda())
print("GPUs:", tf.config.list_physical_devices("GPU"))

Built with CUDA: False
GPUs: []


In [3]:
IMG_SIZE = 64
CHANNELS = 3
BATCH_SIZE = 64
LATENT_DIM = 100
EPOCHS = 50
MAX_IMAGES = 10000          
SAMPLE_DIR = "samples_faces_hf"
os.makedirs(SAMPLE_DIR, exist_ok=True)

In [4]:
from datasets import load_dataset

dataset = load_dataset("nielsr/CelebA-faces")

In [5]:
print("Built with CUDA:", tf.test.is_built_with_cuda())

Built with CUDA: False


In [6]:
# sample image only 10000 images
if len(dataset['train']) > MAX_IMAGES:
    dataset = dataset['train'].shuffle(seed=42).select(range(MAX_IMAGES))
print(len(dataset))

10000


In [7]:
dataset

Dataset({
    features: ['image'],
    num_rows: 10000
})

In [8]:
# Preprocess images
IMG_SIZE_TUPLE = (IMG_SIZE, IMG_SIZE)

def preprocess_batch(batch):
    images = []
    for img in batch["image"]:
        img = img.resize(IMG_SIZE_TUPLE)                
        img = np.array(img, dtype=np.float32)           
        images.append(img)
    images = np.stack(images, axis=0)                   
    images = (images - 127.5) / 127.5                   
    return {"images": images}

dataset = dataset.map(
    preprocess_batch,
    batched=True,
    remove_columns=dataset.column_names
)

In [9]:
dataset

Dataset({
    features: ['images'],
    num_rows: 10000
})

In [10]:
images_list = dataset["images"]           # list of (64,64,3)
train_images = np.stack(images_list, axis=0)
print("train_images shape:", train_images.shape)  # (N,64,64,3)}

train_images shape: (10000, 64, 64, 3)


In [11]:
BUFFER_SIZE = train_images.shape[0]

# Create dataset for training 
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices(train_images)
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.AUTOTUNE)
)


In [12]:
# Create Generator Model
def generator_model():
    model = tf.keras.Sequential(
        [
            layers.Dense(8 * 8 * 512, use_bias=False, input_shape=(LATENT_DIM,)),
            layers.BatchNormalization(),
            layers.LeakyReLU(),

            layers.Reshape((8, 8, 512)),   # (8,8,512)

            # 8x8 -> 16x16
            layers.Conv2DTranspose(
                256, (5, 5), strides=(2, 2), padding="same", use_bias=False
            ),
            layers.BatchNormalization(),
            layers.LeakyReLU(),

            # 16x16 -> 32x32
            layers.Conv2DTranspose(
                128, (5, 5), strides=(2, 2), padding="same", use_bias=False
            ),
            layers.BatchNormalization(),
            layers.LeakyReLU(),

            # 32x32 -> 64x64
            layers.Conv2DTranspose(
                64, (5, 5), strides=(2, 2), padding="same", use_bias=False
            ),
            layers.BatchNormalization(),
            layers.LeakyReLU(),

            # output: 64x64x3
            layers.Conv2DTranspose(
                CHANNELS,
                (5, 5),
                strides=(1, 1),
                padding="same",
                use_bias=False,
                activation="tanh",  # output in [-1,1]
            ),
        ],
        name="Generator",
    )
    return model


In [13]:
# Create Discriminator Model

def discriminator_model():
    model = tf.keras.Sequential(
        [
            layers.Conv2D(
                64,
                (5, 5),
                strides=(2, 2),
                padding="same",
                input_shape=(IMG_SIZE, IMG_SIZE, CHANNELS),
            ),
            layers.LeakyReLU(),
            layers.Dropout(0.3),

            layers.Conv2D(128, (5, 5), strides=(2, 2), padding="same"),
            layers.LeakyReLU(),
            layers.Dropout(0.3),

            layers.Conv2D(256, (5, 5), strides=(2, 2), padding="same"),
            layers.LeakyReLU(),
            layers.Dropout(0.3),

            layers.Flatten(),
            layers.Dense(1),  # from_logits=True
        ],
        name="Discriminator",
    )
    return model

In [14]:
generator = generator_model()
discriminator = discriminator_model()

print(generator.summary())
print(discriminator.summary())

c:\Users\earth\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\earth\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "Generator"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32768)          │     3,276,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 32768)          │       131,072 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 32768)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 8, 8, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose                │ (None, 16, 16, 256)    │     3,276,800 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 16, 16, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_1 (LeakyReLU)       │ (None, 16, 16, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_1              │ (None, 32, 32, 128)    │       819,200 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 32, 32, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_2 (LeakyReLU)       │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_2              │ (None, 64, 64, 64)     │       204,800 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 64, 64, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_3 (LeakyReLU)       │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_3              │ (None, 64, 64, 3)      │         4,800 │
│ (Conv2DTranspose)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,715,264 (29.43 MB)

 Trainable params: 7,648,832 (29.18 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 32, 32, 64)     │         4,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_4 (LeakyReLU)       │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 16, 16, 128)    │       204,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_5 (LeakyReLU)       │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 8, 8, 256)      │       819,456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_6 (LeakyReLU)       │ (None, 8, 8, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 8, 8, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 16384)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │        16,385 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,045,633 (3.99 MB)

 Trainable params: 1,045,633 (3.99 MB)

 Non-trainable params: 0 (0.00 B)

None


In [15]:
# evaluate model
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_logits, fake_logits):
    real_loss = cross_entropy(tf.ones_like(real_logits), real_logits)
    fake_loss = cross_entropy(tf.zeros_like(fake_logits), fake_logits)
    return real_loss + fake_loss

def generator_loss(fake_logits):
    return cross_entropy(tf.ones_like(fake_logits), fake_logits)

generator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

num_examples_to_generate = 16
seed = tf.random.normal([num_examples_to_generate, LATENT_DIM])


In [16]:
# train model
@tf.function
def train_step(real_images):
    # Create noise for generate the fake image
    noise = tf.random.normal([BATCH_SIZE, LATENT_DIM])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        fake_images = generator(noise, training=True)

        real_logits = discriminator(real_images, training=True)
        fake_logits = discriminator(fake_images, training=True)

        g_loss = generator_loss(fake_logits)
        d_loss = discriminator_loss(real_logits, fake_logits)

    gradients_gen = gen_tape.gradient(g_loss, generator.trainable_variables)
    gradients_disc = disc_tape.gradient(d_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_gen, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_disc, discriminator.trainable_variables))

    return g_loss, d_loss


In [17]:
def generate_and_save_images(generator, epoch):
    # sample 1 noise vector (shape: 1×100)
    noise = tf.random.normal([1, LATENT_DIM])

    # generate one fake image
    fake = generator(noise, training=False)[0]   # shape: (64,64,3)

    # convert [-1,1] -> [0,1]
    fake = (fake + 1.0) / 2.0
    fake = tf.clip_by_value(fake, 0.0, 1.0).numpy()

    # save the image
   
    path = os.path.join(SAMPLE_DIR, f"result_image_epoch{epoch}.png")
    plt.imsave(path, fake)
    print(f"Saved one generated image to {path}")


In [18]:
from tqdm import tqdm

g_history = []
d_history = []

def train(dataset, epochs):
    for epoch in range(1, epochs + 1):
        start = time.time()
        total_g, total_d, steps = 0.0, 0.0, 0

        # tqdm โชว์ progress ต่อ batch
        pbar = tqdm(dataset, desc=f"Epoch {epoch}/{epochs}", ncols=80)
        for batch in pbar:
            g_loss, d_loss = train_step(batch)
            total_g += g_loss
            total_d += d_loss
            steps += 1

            # อัปเดตข้อความใน progress bar (โชว์ loss ล่าสุด)
            pbar.set_postfix({
                "G_loss": float(g_loss.numpy()),
                "D_loss": float(d_loss.numpy())
            })

        avg_g = total_g / steps
        avg_d = total_d / steps
        g_history.append(float(avg_g.numpy()))
        d_history.append(float(avg_d.numpy()))

        print(
            f"\nEpoch {epoch}/{epochs} | "
            f"avg G loss: {avg_g:.4f} | avg D loss: {avg_d:.4f} | "
            f"Time: {time.time() - start:.2f} sec"
        )
        generate_and_save_images(generator, epoch)

if __name__ == "__main__":
    train(train_dataset, EPOCHS)
    generator.save_weights("generator_celeba_hf.weights.h5")
    discriminator.save_weights("discriminator_celeba_hf.weights.h5")
    print("Training finished & weights saved.")

Epoch 1/50: 100%|██| 156/156 [03:01<00:00,  1.16s/it, G_loss=0.615, D_loss=1.64]



Epoch 1/50 | avg G loss: 2.6246 | avg D loss: 0.6854 | Time: 181.12 sec
Saved one generated image to samples_faces_hf\result_image_epoch1.png


Epoch 2/50: 100%|██| 156/156 [02:58<00:00,  1.14s/it, G_loss=0.625, D_loss=1.34]



Epoch 2/50 | avg G loss: 0.7342 | avg D loss: 1.3611 | Time: 178.53 sec
Saved one generated image to samples_faces_hf\result_image_epoch2.png


Epoch 3/50: 100%|██| 156/156 [02:52<00:00,  1.10s/it, G_loss=0.711, D_loss=1.48]


...


Epoch 48/50 | avg G loss: 0.9110 | avg D loss: 1.2403 | Time: 183.83 sec
Saved one generated image to samples_faces_hf\result_image_epoch48.png


Epoch 49/50: 100%|██| 156/156 [03:04<00:00,  1.18s/it, G_loss=0.935, D_loss=1.3]



Epoch 49/50 | avg G loss: 0.9069 | avg D loss: 1.2384 | Time: 184.22 sec
Saved one generated image to samples_faces_hf\result_image_epoch49.png


Epoch 50/50: 100%|█| 156/156 [03:03<00:00,  1.18s/it, G_loss=0.978, D_loss=1.22]


Epoch 50/50 | avg G loss: 0.9026 | avg D loss: 1.2466 | Time: 183.69 sec
Saved one generated image to samples_faces_hf\result_image_epoch50.png
Training finished & weights saved.


In [ ]:
# generator = generator_model()
# discriminator = discriminator_model()

In [ ]:
# generator.load_weights("generator_celeba_hf.weights.h5")
# discriminator.load_weights("discriminator_celeba_hf.weights.h5")